In [ ]:
import os
import sys

ENDSWITH = 'OCR'

NOTEBOOK_DIR = os.getcwd()

if not NOTEBOOK_DIR.endswith(ENDSWITH):
    raise ValueError(f"Not in correct dir, expect end with {ENDSWITH}, but got {NOTEBOOK_DIR} instead")

BASE_DIR = os.path.abspath(os.path.join(NOTEBOOK_DIR, '..', '..', '..', '..'))
print(BASE_DIR)

sys.path.insert(0, os.path.join(BASE_DIR, 'code'))

In [ ]:
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")


In [ ]:
import importlib
import MangaOCREvaluator
importlib.reload(MangaOCREvaluator)

from MangaOCREvaluator import MangaOCREvaluator
from pipeline.OCRModels.MangaOCRModel import MangaOCRModel

# Import process_manga from link_segment_to_text.py
from pathlib import Path
import utils.link_segment_to_text as link_segment
importlib.reload(link_segment)
from utils.link_segment_to_text import process_manga

In [ ]:
# Paths configuration
manga_name = "AisazuNihaIrarenai"

# Input paths
json_segmentation_path = Path(BASE_DIR) / 'data' / 'MangaSegmentation' / 'jsons_processed' / f'{manga_name}.json'
xml_path = Path(BASE_DIR) / 'data' / 'Manga109_released_2023_12_07' / 'annotations' / f'{manga_name}.xml'
images_dir = os.path.join(BASE_DIR, 'data', 'Manga109_released_2023_12_07', 'images', manga_name)

# Output path
json_output_path = Path(BASE_DIR) / 'data' / 'MangaOCR' / 'jsons_processed' / f'{manga_name}.json'

# Process: Link segmentation bubbles with text annotations from XML
# This creates JSON with both bubble_bbox (from segmentation) and text_bbox (from XML)
process_manga(json_segmentation_path, xml_path, json_output_path)

print(f"\nOutput JSON: {json_output_path}")

In [ ]:
evaluator = MangaOCREvaluator(device=device)

ocr_model = MangaOCRModel()

metrics = evaluator.compare_bbox_types(
    ocr_model=ocr_model,
    json_path=str(json_output_path),
    images_dir=images_dir,
    batch_size=1,
    verbose=False,
    max_images=20
)